## Tratamento do PPA por regionalização

Nesse notebook são tratados os dados do Plano Plurianual por regionalização (2022 - 2025) da Prefeitura Municipal de São Paulo

In [1]:
import pandas as pd
from utils.load_csv import load_csv
from utils.load_shp import load_shp
from utils.save_csv import save_csv

In [2]:
df_ppa_reg = load_csv("ppa_reg.csv")

#### Criação "id"

Cria variável "id" concatenando variáveis de código da base de dados para "chave" do merge

In [3]:
df_ppa_reg["id"] = df_ppa_reg[["codigo orgao",
                               "codigo unidade",
                               "codigo programa",
                               "codigo acao"]].apply(lambda x: "_".join(x.astype(str)), axis = 1)

In [4]:
cols = ["id"] + [col for col in df_ppa_reg.columns if col != "id"]

df_ppa_reg = df_ppa_reg[cols]

#### Transformação das variáveis de planejamento de recurso

Transforma as variáveis de planejamento de resurso da base de dados de string para float

In [5]:
cols_plan_recurso = ["valor 2022",
                     "valor 2023",
                     "valor 2024",
                     "valor 2025"]

In [6]:
for col in cols_plan_recurso:
    df_ppa_reg[col] = (df_ppa_reg[col]
                         .str.replace(".", "")
                         .str.replace(",", ".")
                         .astype(float))

#### Filtro regionalizável

Filtra a base de dados para conter apenas despesas regionalizáveis

In [7]:
supra_subprefs = ["Supra Subprefeitura",
                  "Supra Subprefeitura Leste",
                  "Supra Subprefeitura Oeste",
                  "Supra Subprefeitura Norte",
                  "Supra Subprefeitura Sul",
                  "Supra Subprefeitura Centro"]

In [8]:
df_ppa_reg = df_ppa_reg[~df_ppa_reg["descricao prefeitura regional"].isin(supra_subprefs)]

#### Normalização das subprefeituras

Trata os nomes das subprefreituras na base de dados para estar igual aos nomes do shapefile das subprefeituras

In [9]:
mapper = {"Subprefeitura Sé": "SE",
          "Subprefeitura Vila Mariana": "VILA MARIANA",
          "Subprefeitura Santana/Tucuruvi": "SANTANA-TUCURUVI",
          "Subprefeitura Campo Limpo": "CAMPO LIMPO",
          "Subprefeitura Itaim Paulista": "ITAIM PAULISTA",
          "Subprefeitura Pirituba/Jaraguá": "PIRITUBA-JARAGUA",
          "Subprefeitura Sapopemba": "SAPOPEMBA",
          "Subprefeitura Capela do Socorro": "CAPELA DO SOCORRO",
          "Subprefeitura Lapa": "LAPA",
          "Subprefeitura Jaçanã/Tremembé": "JACANA-TREMEMBE",
          "Subprefeitura Penha": "PENHA",
          "Subprefeitura São Miguel Paulista": "SAO MIGUEL",
          "Subprefeitura Perus/Anhanguera": "PERUS",
          "Subprefeitura Butantã": "BUTANTA",
          "Subprefeitura de Guaianases": "GUAIANASES",
          "Subprefeitura São Mateus": "SAO MATEUS",
          "Subprefeitura Cidade Ademar": "CIDADE ADEMAR",
          "Subprefeitura de Vila Prudente": "VILA PRUDENTE",
          "Subprefeitura Ipiranga": "IPIRANGA",
          "Subprefeitura Pinheiros": "PINHEIROS",
          "Subprefeitura Casa Verde/Cachoeirinha": "CASA VERDE-CACHOEIRINHA",
          "Subprefeitura Freguesia/Brasilândia": "FREGUESIA-BRASILANDIA",
          "Subprefeitura M'Boi Mirim": "M BOI MIRIM",
          "Subprefeitura Vila Maria/Vila Guilherme": "VILA MARIA-VILA GUILHERME",
          "Subprefeitura Aricanduva/Formosa/Carrão": "ARICANDUVA-FORMOSA-CARRAO",
          "Subprefeitura Santo Amaro": "SANTO AMARO",
          "Subprefeitura Mooca": "MOOCA",
          "Subprefeitura Ermelino Matarazzo": "ERMELINO MATARAZZO",
          "Subprefeitura Cidade Tiradentes": "CIDADE TIRADENTES",
          "Subprefeitura Itaquera": "ITAQUERA",
          "Subprefeitura Jabaquara": "JABAQUARA",
          "Subprefeitura Parelheiros": "PARELHEIROS"}

In [10]:
df_ppa_reg.loc[:, "descricao prefeitura regional"] = df_ppa_reg["descricao prefeitura regional"].map(mapper)

In [11]:
pd.set_option("display.max_columns", None)

df_ppa_reg.sample(10)

,id,codigo orgao,descricao orgao,codigo unidade,descricao unidade,codigo programa,descricao programa,codigo acao,descricao acao,codigo da,descricao da,codigo meta,descricao meta,codigo regiao,descricao regiao,codigo prefeitura regional,descricao prefeitura regional,codigo distrito,descricao distrito,codigo sequencial da,codigo produto,descricao produto,descricao unidade medida,quantidade 2022,valor 2022,quantidade 2023,valor 2023,quantidade 2024,valor 2024,quantidade 2025,valor 2025
5476,25_10_3001_6410,25,Secretaria Municipal de Cultura,10,Gabinete do Secretário,3001,Acesso à Cultura,6410,Carnaval de Rua de São Paulo,777.03.10.00.001,Subprefeitura Butantã,777,Despesa Regionalizável,3,Oeste,10,BUTANTA,0,Supra-Distrital,1,2904,Atividades culturais realizadas,ações,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
1841,16_10_3025_2876,16,Secretaria Municipal de Educação,10,Gabinete do Secretário,3025,Proteção e Desenvolvimento Integral na Primeir...,2876,Manutenção e Operação de Centros Municipais de...,777.04.16.00.001,Subprefeitura Cidade Ademar,777,Despesa Regionalizável,4,Sul,16,CIDADE ADEMAR,0,Supra-Distrital,1,2619,CEMEIs em funcionamento,unida,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
3828,25_10_3001_2025,25,Secretaria Municipal de Cultura,10,Gabinete do Secretário,3001,Acesso à Cultura,2025,Manutenção e Operação da Biblioteca Mario de A...,777.02.32.00.001,Subprefeitura Sapopemba,777,Despesa Regionalizável,2,Leste,32,SAPOPEMBA,0,Supra-Distrital,1,2897,Atividades Culturais da Biblioteca Mario de An...,ações,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
4097,25_10_3001_5406,25,Secretaria Municipal de Cultura,10,Gabinete do Secretário,3001,Acesso à Cultura,5406,"Ampliação, Reforma e Requalificação de Equipam...",777.01.03.00.001,Subprefeitura Freguesia/Brasilândia,777,Despesa Regionalizável,1,Norte,3,FREGUESIA-BRASILANDIA,0,Supra-Distrital,1,2960,"Ampliação, Reforma e Requalificação de Equipam...",unida,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
2781,20_10_3009_1097,20,Secretaria Municipal de Mobilidade e Trânsito,10,Gabinete do Secretário,3009,Melhoria da Mobilidade Urbana Universal,1097,"Construção de Ciclovias, Ciclofaixas e Ciclorr...",777.01.02.00.001,Subprefeitura Pirituba/Jaraguá,777,Despesa Regionalizável,1,Norte,2,PIRITUBA-JARAGUA,0,Supra-Distrital,1,2723,"Extensão de ciclovias, ciclofaixas e ciclorrot...",quilô,"37,00",85.0,"40,00",1637907.0,42,1735222.0,47,1871925.0
786,14_10_3002_3357,14,Secretaria Municipal de Habitação,10,Gabinete do Secretário,3002,Acesso à Moradia Adequada,3357,Urbanização de Favelas,777.02.30.00.001,Subprefeitura São Mateus,777,Despesa Regionalizável,2,Leste,30,SAO MATEUS,0,Supra-Distrital,1,2645,Famílias beneficiadas com obras de urbanização...,famíl,"0,00",10083535.0,"30,00",15004109.0,4.298,8166077.0,0,5048530.0
5323,25_10_3001_6393,25,Secretaria Municipal de Cultura,10,Gabinete do Secretário,3001,Acesso à Cultura,6393,Programação de Atividades Culturais do Centro ...,777.02.30.00.001,Subprefeitura São Mateus,777,Despesa Regionalizável,2,Leste,30,SAO MATEUS,0,Supra-Distrital,1,2904,Atividades culturais realizadas,ações,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
9300,91_10_3002_3354,91,Fundo Municipal de Habitação,10,Fundo Municipal de Habitação,3002,Acesso à Moradia Adequada,3354,Construção de Unidades Habitacionais,777.04.16.00.001,Subprefeitura Cidade Ademar,777,Despesa Regionalizável,4,Sul,16,CIDADE ADEMAR,0,Supra-Distrital,1,2648,Unidades Habitacionais entregues por meio de p...,unida,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
10763,98_22_3009_5100,98,Fundo de Desenvolvimento Urbano,22,FUNDURB - Secretaria Municipal de Infraestrutu...,3009,Melhoria da Mobilidade Urbana Universal,5100,Intervenções no Sistema Viário,777.02.32.00.001,Subprefeitura Sapopemba,777,Despesa Regionalizável,2,Leste,32,SAPOPEMBA,0,Supra-Distrital,1,2659,"Intervenções (construção, requalificação, etc)...",%,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
7554,61_10_3008_2367,61,Subprefeitura Penha,10,Administração da Subprefeitura,3008,Gestão dos Riscos e Promoção da Resiliência a ...,23

In [12]:
save_csv(df_ppa_reg, "ppa_reg.csv")

Base salva em data\ppa_reg.csv
